In [1]:
import pandas as pd
import numpy as np
import sklearn
import nltk
import re
import math

In [2]:
data = pd.read_csv("hn2018_2019.csv")

In [3]:
def dater(datetim,integer):
    date,time = datetim.split(" ")
    year,month,day = date.split("-")
    hour,minute,second = time.split(":")
    second = int(second)+ int(minute)*60 + int(hour)*60*60
    if integer==1:
        return year
    elif integer==2:
        return month
    elif integer==3:
        return day
    else:
        return second

In [4]:
data["year"] = data['Created At'].map(lambda x: dater(x,1))
data["month"] = data['Created At'].map(lambda x: dater(x,2))
data["day"] = data['Created At'].map(lambda x: dater(x,3))
data["second"] = data['Created At'].map(lambda x: dater(x,4))

In [5]:
df_2018 = data[data["year"]=='2018'][['Title','Post Type']].copy()

In [6]:
df_2018['Title'] = df_2018['Title'].map(lambda x:x.lower())

## Adding a column of tokenized words in the dataframe

In [7]:
df_2018['tokenized_title'] = df_2018['Title'].map(lambda x:re.split('\[\^a-zA-Z\]',x)[0].split())

In [8]:
data.head()

,Unnamed: 0,Object ID,Title,Post Type,Author,Created At,URL,Points,Number of Comments,year,month,day,second
0,2294635,16043786,Top Hacker News Books in 2017,story,0x54MUR41,2018-01-01 00:06:16,https://hackernewsbooks.com/year/2017,1,0.0,2018,01,01,376
1,2294636,16043817,Beijing enjoys best winter air quality in five...,story,gpetukhov,2018-01-01 00:15:13,http://www.scmp.com/news/china/policies-politi...,2,0.0,2018,01,01,913
2,2294637,16043825,Ask HN: Any domain name registrars that don't ...,ask_hn,glockenspielen,2018-01-01 00:16:54,NaN,1,2.0,2018,01,01,1014
3,2294638,16043845,Controversial Therapy Has Led to Death Threats...,story,cpncrunch,2018-01-01 00:23:05,https://www.buzzfeed.com/tomchivers/inside-the...,2,0.0,2018,01,01,1385
4,2294639,16043851,Ruby 3x3 – Ruby 3 Will Be 3 Times Faster – Wha...,story,geraldbauer,2018-01-01 00:25:05,https://planetruby.github.io/advent2017/ruby3x3,2,0.0,2018,01,01,1505


In [9]:
df_2018.head()

,Title,Post Type,tokenized_title
0,top hacker news books in 2017,story,"[top, hacker, news, books, in, 2017]"
1,beijing enjoys best winter air quality in five...,story,"[beijing, enjoys, best, winter, air, quality, ..."
2,ask hn: any domain name registrars that don't ...,ask_hn,"[ask, hn:, any, domain, name, registrars, that..."
3,controversial therapy has led to death threats...,story,"[controversial, therapy, has, led, to, death, ..."
4,ruby 3x3 – ruby 3 will be 3 times faster – wha...,story,"[ruby, 3x3, –, ruby, 3, will, be, 3, times, fa..."


In [10]:
list(np.unique(data['Post Type']))

['ask_hn', 'poll', 'show_hn', 'story']

## Calculating frequency of each word and given their conditional post type

In [10]:
def input_phrase(phrase,typ,dict_words,labels):
    for i in phrase:
        if i not in dict_words:
            dict_words[i] = {}
            for j in labels:
                #print(j)
                dict_words[i][j] = 0
        dict_words[i][typ] += 1
    return dict_words

In [ ]:
d = {}
totalNumberOfWords = 0
uniqueWords = 0
for i in range(len(df_2018['tokenized_title'])):
    d = input_phrase(df_2018['tokenized_title'][i],df_2018['Post Type'][i],d,list(np.unique(data['Post Type'])))
    totalNumberOfWords += len(df_2018['tokenized_title'][i])
    if len(d)%10000==0:
        print("Au travail")

In [42]:
ateendre = pd.DataFrame(d)

In [45]:
ateendre[ateendre<20] = 0
ateendre['github']

ask_hn       40
poll          0
show_hn     127
story      1028
Name: github, dtype: int64

In [12]:
words_2018 = pd.DataFrame(d)
words_2018 = words_2018.transpose()
delta = 0.5
uniqueWords = len(words_2018)
absentWordConditionalProbability = {}
for className in list(np.unique(data['Post Type'])):
    wordsPerClass = np.sum(words_2018[className])
    conditionalClassLabel = 'prob_' + className
    words_2018[conditionalClassLabel] = words_2018[className].map(lambda x: (int(x) + delta)/( wordsPerClass + delta*uniqueWords ))    
    absentWordConditionalProbability[conditionalClassLabel] = delta/( wordsPerClass + delta*uniqueWords )
    
writeWords2018 = words_2018.copy()
words_2018 = words_2018.transpose()

In [13]:
writeWords2018.index.name = 'TokenName'
writeWords2018 = writeWords2018.reset_index()
writeWords2018 = writeWords2018.sort_values(by ='TokenName')
writeWords2018 = writeWords2018.reset_index()
cols = ['TokenName', 'story', 'prob_story', 'ask_hn', 'prob_ask_hn', 'show_hn', 'prob_show_hn', 'poll', 'prob_poll']
writeWords2018 = writeWords2018[cols]
writeWords2018.index += 1
filename = 'model-2018.txt'
writeWords2018.to_csv(filename, header = None, index = True, sep = ' ', mode = 'w')
with open(filename, 'r') as f:
    lines = f.readlines()
lines = [line.replace(' ', '  ') for line in lines]
with open(filename, 'w') as f:
    f.writelines(lines)

In [14]:
words_2018.head(10)

,!,!!,!!`?,!=,!==,"""+=""","""12","""@""","""[github","""a11y""",...,𝗿𝗶𝗰𝗵,𝘵𝘦𝘹𝘵,𝚑𝚝𝚖𝚕,🆙,🇨🇳,🇺🇸,🧐,🧙,🧙‍️,🧞‍️
ask_hn,1.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
poll,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
show_hn,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
story,1.000000e+00,1.000000e+00,1.000000e+00,6.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
prob_ask_hn,7.307840e-06,2.435947e-06,2.435947e-06,0.000002,2.435947e-06,2.435947e-06,2.435947e-06,2.435947e-06,2.435947e-06,7.307840e-06,...,2.435947e-06,2.435947e-06,2.435947e-06,2.435947e-06,2.435947e-06,2.435947e-06,2.435947e-06,2.435947e-06,2.435947e-06,2.435947e-06
prob_poll,6.521030e-06,6.521030e-06,6.521030e-06,0.000007,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06,...,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06
prob_show_hn,2.824045e-06,2.824045e-06,2.824045e-06,0.000003,2.824045e-06,2.824045e-06,2.824045e-06,2.824045e-06,2.824045e-06,2.824045e-06,...,2.824045e-06,2.824045e-06,2.824045e-06,2.824045e-06,2.824045e-06,2.824045e-06,2.824045e-06,2.824045e-06,2.824045e-06,8.472135e-06
prob_story,7.051351e-07,7.051351e-07,7.051351e-07,0.000003,7.051351e-07,7.051351e-07,7.051351e-07,7.051351e-07,7.051351e-07,2.350450e-07,...,7.051351e-07,7.051351e-07,7.051351e-07,7.051351e-07,7.051351e-07,7.051351e-07,7.051351e-07,7.051351e-07,7.051351e-07,2.350450e-07


## PART 2

In [15]:
priorProbabilities = {'prob_story':0,'prob_ask_hn':0,'prob_show_hn':0,'prob_poll':0}
unique, counts = np.unique(df_2018['Post Type'], return_counts=True)
for index in range(len(unique)):
    priorProbabilities['prob_' + unique[index]] = counts[index]/np.sum(counts)
print(priorProbabilities)

{'prob_story': 0.917831909047913, 'prob_poll': 9.025889862481542e-05, 'prob_ask_hn': 0.04516194251591264, 'prob_show_hn': 0.036915889537549505}


In [16]:
conditionalProbabilities = words_2018.drop(['story', 'poll', 'show_hn', 'ask_hn'])
conditionalProbabilities.head(10)

,!,!!,!!`?,!=,!==,"""+=""","""12","""@""","""[github","""a11y""",...,𝗿𝗶𝗰𝗵,𝘵𝘦𝘹𝘵,𝚑𝚝𝚖𝚕,🆙,🇨🇳,🇺🇸,🧐,🧙,🧙‍️,🧞‍️
prob_ask_hn,7.307840e-06,2.435947e-06,2.435947e-06,0.000002,2.435947e-06,2.435947e-06,2.435947e-06,2.435947e-06,2.435947e-06,7.307840e-06,...,2.435947e-06,2.435947e-06,2.435947e-06,2.435947e-06,2.435947e-06,2.435947e-06,2.435947e-06,2.435947e-06,2.435947e-06,2.435947e-06
prob_poll,6.521030e-06,6.521030e-06,6.521030e-06,0.000007,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06,...,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06,6.521030e-06
prob_show_hn,2.824045e-06,2.824045e-06,2.824045e-06,0.000003,2.824045e-06,2.824045e-06,2.824045e-06,2.824045e-06,2.824045e-06,2.824045e-06,...,2.824045e-06,2.824045e-06,2.824045e-06,2.824045e-06,2.824045e-06,2.824045e-06,2.824045e-06,2.824045e-06,2.824045e-06,8.472135e-06
prob_story,7.051351e-07,7.051351e-07,7.051351e-07,0.000003,7.051351e-07,7.051351e-07,7.051351e-07,7.051351e-07,7.051351e-07,2.350450e-07,...,7.051351e-07,7.051351e-07,7.051351e-07,7.051351e-07,7.051351e-07,7.051351e-07,7.051351e-07,7.051351e-07,7.051351e-07,2.350450e-07


In [38]:
def getConditionalProbability(word,className,absentWordConditionalProbability,conditionalProbabilities):
    try:
        return conditionalProbabilities[word][className]
    except:
        return absentWordConditionalProbability[className]

In [39]:
def generateScore(arrayOfTokenizedTitle,className,priorProbabilities,absentWordConditionalProbability,conditionalProbabilities):
    score = np.zeros(len(arrayOfTokenizedTitle)) + math.log10(priorProbabilities[className])
    for index in range(len(arrayOfTokenizedTitle)):
        wordsList = arrayOfTokenizedTitle[index]
        for word in wordsList:
            score[index] += math.log10(getConditionalProbability(word,className,absentWordConditionalProbability,conditionalProbabilities))
    return score

In [11]:
testData = data[data["year"]=='2019'][['Title','Post Type']].copy()
testData['Title'] = testData['Title'].map(lambda x:x.lower())
testData['tokenized_title'] = testData['Title'].map(lambda x:re.split('\[\^a-zA-Z\]',x)[0].split())
testData = testData.reset_index()

In [20]:
testData.head()

,index,Title,Post Type,tokenized_title
0,276981,the tech that was fixed in 2018 and the tech t...,story,"[the, tech, that, was, fixed, in, 2018, and, t..."
1,276982,why is the google podcasts app failing so hard?,story,"[why, is, the, google, podcasts, app, failing,..."
2,276983,doing dishes is the worst,story,"[doing, dishes, is, the, worst]"
3,276984,setting up a mongodb replica set with docker a...,story,"[setting, up, a, mongodb, replica, set, with, ..."
4,276985,history favors co-operation and non-zero sum g...,story,"[history, favors, co-operation, and, non-zero,..."


In [21]:
for className in priorProbabilities.keys():
    testData[className] = generateScore(testData['tokenized_title'],className)
testData.head()

,index,Title,Post Type,tokenized_title,prob_story,prob_poll,prob_ask_hn,prob_show_hn
0,276981,the tech that was fixed in 2018 and the tech t...,story,"[the, tech, that, was, fixed, in, 2018, and, t...",-39.125383,-73.781355,-45.343107,-49.471938
1,276982,why is the google podcasts app failing so hard?,story,"[why, is, the, google, podcasts, app, failing,...",-28.429325,-49.284300,-30.506939,-35.941511
2,276983,doing dishes is the worst,story,"[doing, dishes, is, the, worst]",-16.393942,-28.541565,-18.880287,-21.777901
3,276984,setting up a mongodb replica set with docker a...,story,"[setting, up, a, mongodb, replica, set, with, ...",-47.162166,-79.444160,-55.254940,-52.654563
4,276985,history favors co-operation and non-zero sum g...,story,"[history, favors, co-operation, and, non-zero,...",-31.325273,-40.344296,-34.342296,-33.529141


In [22]:
columnNamesExchange = {}
for className in ['ask_hn', 'poll', 'show_hn', 'story']:
    columnNamesExchange['prob_' + className] = className
dfObj = testData[['prob_ask_hn', 'prob_poll', 'prob_show_hn', 'prob_story']].copy()
dfObj = dfObj.rename(columns=columnNamesExchange)
dfObj['predicted'] = dfObj.idxmax(axis=1)
dfObj = pd.concat([dfObj, testData[['Title', 'Post Type']]], axis=1)
dfObj['comparision'] = (dfObj['predicted'] == dfObj['Post Type'])
cols = ['Title', 'story', 'ask_hn', 'show_hn', 'poll', 'Post Type', 'comparision']
dfObj = dfObj[cols]
dfObj.head()

,Title,story,ask_hn,show_hn,poll,Post Type,comparision
0,the tech that was fixed in 2018 and the tech t...,-39.125383,-45.343107,-49.471938,-73.781355,story,True
1,why is the google podcasts app failing so hard?,-28.429325,-30.506939,-35.941511,-49.284300,story,True
2,doing dishes is the worst,-16.393942,-18.880287,-21.777901,-28.541565,story,True
3,setting up a mongodb replica set with docker a...,-47.162166,-55.254940,-52.654563,-79.444160,story,True
4,history favors co-operation and non-zero sum g...,-31.325273,-34.342296,-33.529141,-40.344296,story,True


In [23]:
filename = 'result.txt'
dfObj.to_csv(filename, header = None, index = False, sep = ' ', mode = 'w')
with open(filename, 'r') as f:
    lines = f.readlines()
lines = [line.replace(' ', '  ') for line in lines]
with open(filename, 'w') as f:
    f.writelines(lines)

In [49]:
check = dfObj.comparision.value_counts()

In [53]:
dict(check).keys()

dict_keys([False, True])

In [54]:
accuracy = check[True]/(check[True]+check[False])

In [55]:
accuracy

0.9557628539153883

In [42]:
def model_making(df_train, df_test, labels, delta, file=True, stopwords = None, blocks = None, frequency = None):
    df_train['Title'] = df_train['Title'].map(lambda x:x.lower())
    df_test['Title'] = df_test['Title'].map(lambda x:x.lower())
    if blocks:
        df_train['tokenized_title'] = df_train['Title'].map(lambda x:[i for i in re.split('\[\^a-zA-Z\]',x)[0].split() if (len(i)>blocks[0] and len(i)<blocks[1])])
        df_test['tokenized_title'] = df_test['Title'].map(lambda x:[i for i in re.split('\[\^a-zA-Z\]',x)[0].split() if (len(i)>blocks[0] and len(i)<blocks[1])])
    elif stopwords:
        df_train['tokenized_title'] = df_train['Title'].map(lambda x:[i for i in re.split('\[\^a-zA-Z\]',x)[0].split() if i not in stopwords])
        df_test['tokenized_title'] = df_test['Title'].map(lambda x:[i for i in re.split('\[\^a-zA-Z\]',x)[0].split() if i not in stopwords])
    else:
        df_train['tokenized_title'] = df_train['Title'].map(lambda x:re.split('\[\^a-zA-Z\]',x)[0].split())
        df_test['tokenized_title'] = df_test['Title'].map(lambda x:re.split('\[\^a-zA-Z\]',x)[0].split())
    d = {}
    totalNumberOfWords = 0
    uniqueWords = 0
    print("Training Started ")
    for i in range(len(df_train['tokenized_title'])):
        d = input_phrase(df_train['tokenized_title'][i],df_train['Post Type'][i],d,labels)
        totalNumberOfWords += len(df_train['tokenized_title'][i])
    df_train = pd.DataFrame(d)
    if frequency:
        df_train[df_train<frequency] = 0
    df_train = df_train.transpose()
    uniqueWords = len(df_train)
    absentWordConditionalProbability = {}
    list_classes = []
    for className in labels:
        wordsPerClass = np.sum(df_train[className])
        conditionalClassLabel = 'prob_' + className
        df_train[conditionalClassLabel] = df_train[className].map(lambda x: (int(x) + delta)/( wordsPerClass + delta*uniqueWords ))    
        absentWordConditionalProbability[conditionalClassLabel] = delta/( wordsPerClass + delta*uniqueWords )
        list_classes.append(className)
        list_classes.append(conditionalClassLabel)
    writeWords = df_train.copy()
    words_train = df_train.transpose()
    if file:
        print("Writing the training model to file ")
        writeWords.index.name = 'TokenName'
        writeWords = writeWords.reset_index()
        writeWords = writeWords.sort_values(by ='TokenName')
        writeWords = writeWords.reset_index()
        cols = ['TokenName'] + list_classes
        print(cols)
        writeWords = writeWords[cols]
        writeWords.index += 1
        filename = 'model-2018.txt'
        writeWords.to_csv(filename, header = None, index = True, sep = ' ', mode = 'w')
        with open(filename, 'r') as f:
            lines = f.readlines()
        lines = [line.replace(' ', '  ') for line in lines]
        with open(filename, 'w') as f:
            f.writelines(lines)
    print("Testing started")
    conditionalProbabilities = words_train.drop(labels)
    unique, counts = np.unique(df_2018['Post Type'], return_counts=True)
    priorProbabilities = {}
    for index in range(len(unique)):
        priorProbabilities['prob_' + unique[index]] = counts[index]/np.sum(counts)
    for className in list(priorProbabilities.keys()):
        df_test[className] = generateScore(df_test['tokenized_title'],className,priorProbabilities,absentWordConditionalProbability,conditionalProbabilities)
    columnNamesExchange = {}
    for className in labels:
        columnNamesExchange['prob_' + className] = className
    dfObj = testData[list(priorProbabilities.keys())].copy()
    dfObj = dfObj.rename(columns=columnNamesExchange)
    dfObj['predicted'] = dfObj.idxmax(axis=1)
    dfObj = pd.concat([dfObj, testData[['Title', 'Post Type']]], axis=1)
    dfObj['comparision'] = (dfObj['predicted'] == dfObj['Post Type'])
    cols = ['Title'] + labels +['Post Type', 'comparision']
    #cols = ['Title', 'story', 'ask_hn', 'show_hn', 'poll', 'Post Type', 'comparision']
    dfObj = dfObj[cols]
    if file:
        print("Writing a Test file")
        filename = 'result.txt'
        dfObj.to_csv(filename, header = None, index = False, sep = ' ', mode = 'w')
        with open(filename, 'r') as f:
            lines = f.readlines()
        lines = [line.replace(' ', '  ') for line in lines]
        with open(filename, 'w') as f:
            f.writelines(lines)
    check = dfObj.comparision.value_counts()
    accuracy = check[True]/(check[True]+check[False])
    print("Training Accuracy :",round(accuracy,2))

In [ ]:
model_making(df_2018,testData,list(np.unique(df_2018['Post Type'])),0.5,blocks = [2,9])

Training Started 
